# Training BData HuCLES (Human Confidence Level Evaluation System)


### - Classification of Confidence Level

<img src="CLevels.png">


# Implementation -

## 1. Getting BData

In [ ]:
# Get video frames as images
# !cd ./code_scripts/
# python vid_to_img.py


##### After this, Crop the required portion from images and paste it in respective folders(test, train, val), then run the following scripts

In [ ]:
# Generate Body dataset csv file
# python genBData.py
# !cd ..


In [ ]:
import tensorflow as tf


## 1. Loading the data
Here, we load the dataset and split it into training and testing data

In [1]:
from code_scripts import prepareFData as ppFD

Using TensorFlow backend.


In [2]:
dataset_path = "./body/bdata/dataset.csv"
# noEntry = 1086
train_data, train_labels, test_data, test_labels = ppFD.load_data(dataset_path, nEntries=1086, imgShape=(480, 480))

print("Number of images in Training set:", len(train_data))
print("Number of images in Test set:", len(test_data))

Number of images in Training set: 1046
Number of images in Test set: 40


## 2. Defining and Deploying CNN for training the model
We have built ourselves a Convolutional Neural Network which has convolutional layers armed with filters that extract features out of images.

We have used Keras to deploy our neural network and also have used Dropout layers frequently since Dropout layers inhibit overfitting by randomly dropping out units from the neural network. 

We will use 20 percent of the training data as validation data.

In [3]:
# CNN model package
from code_scripts import modelCNN as md

In [4]:
# Hyperparameters
epochs = 1
batch_size = 2
learning_rate = 0.00001

# defining model
model = md.createCNN(learning_rate, img_ht=480, img_wd=480)

# saving callbacks name
logName = 'UP_fcnn20_ep{1}'

# Callbacks
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=6, mode='auto')
checkpointer = ModelCheckpoint('./model/upperbody/bweights_ep{epoch:03d}_val_loss{val_loss:.3f}.h5', monitor='val_loss', verbose=1, save_best_only=True)
tB = TensorBoard(log_dir='./logs/B000{}'.format(logName))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 478, 478, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 478, 478, 64)      36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 478, 478, 64)      256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 478, 478, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 478, 478, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 239, 239, 64)      0         
______________________________________

In [5]:
# training(or compiling/deploying) the model
model = md.compileCNN(model, train_data, train_labels, epochs, batch_size, lr_reducer, checkpointer, early_stopper, tB)




Train on 836 samples, validate on 210 samples


Epoch 1/1
836/836 [==============================] - 3852s 5s/step - loss: 7.8103 - accuracy: 0.1854 - val_loss: 2.1791 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 2.17909, saving model to ./model/upperbody/bweights_ep001_val_loss2.179.h5



## 3. Testing the trained model

In [ ]:
md.predictCNN(model, test_data, test_labels)

## 4. Saving the trained and tested model

In [1]:
DIR = './model/upperbody/'
md.saveCNN(model)

NameError: name 'md' is not defined

## 5. Evaluating our trained model on Video/Images

In [1]:
!python evaluateVideo.py

Loaded model from disk


Using TensorFlow backend.
T:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
T:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
T:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
T:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1type

#### Screenshot of video prediction output
<img src="opss.png">

##### Sad image output
<img src="OutputImg/sadop.jpg">

##### Angry image output
<img src="OutputImg/angryop.jpg">

##### Too happy image output
<img src="OutputImg/happyop.jpg">

##### Confident image output
<img src="OutputImg/confiop.jpg">